In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import openslide
from io import BytesIO
import os
import numpy as np
import matplotlib.pyplot as plt
import sys
import time
import glob
import pickle
%matplotlib inline

In [ ]:
df = pd.read_csv(r"/content/training_data_classification_labels.csv")
column = "CPM_RadPath_2019_ID"
class_name = "class"
input_path = r'/content/drive/My Drive/Pathology'
p_step=1
patch_size = 512
sdf = 1
down_size = int(patch_size/sdf)
file_size_dict = {}

In [ ]:
for i in range(221):
    img_path = df[column][i]
    biopsy_img = openslide.Openslide(os.path.join(input_path,img_path+".tiff"))
    biopsy_ldim = biopsy_img.level_dimensions[0]
    x_step = int(biopsy_ldim[0]/patch_size)
    y_step = int(biopsy_ldim[1]/patch_size)
    file_size_dict.clear()
    top_patches = int(0.0016*x_step*y_step*3)
    if top_patches < 36:
        top_patches = 36
    
    for j in range(0,x_step,p_step):
        for k in range(0,y_step,p_step):
            biopsy_img0 = biopsy_img.read_region((i*patch_size,j*patch_size), 0, (patch_size,patch_size))
            mem_file = BytesIO()
            biopsy_img0.save(mem_file, 'png')
            mem_file_size = mem_file.tell()
            file_size_dict[(i*patch_size,j*patch_size)] = mem_file_size
    
    sorted_fsd = sorted(file_size_dict.item(), reverse=True)
    patch_list = open(os.path.join(input_path,"Output",img_path,"patch_list.pkl"), "wb")
    pickle.dump(sorted_fsd, patch_list)
    patch_list.close()
    for j in range(top_patches, len(sorted_fsd)):
        biopsy_img0 = biopsy_img.read_region(sorted_fsd[i][0], 0, (patch_size,patch_size))
        biopsy_img0.save(os.path.join(input_path,"Output",img_path,img_path+"_"+str(sorted_fsd[i][0][0])+"_"+str(sorted_fsd[i][0][1])+".png", "PNG"))